In [64]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

# 구글드라이브 연결
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [65]:
# 데이터파일 불러오기
df = pd.read_csv('/gdrive/My Drive/3-2_machine_learning/image-10k.csv', header=None)

X = df.iloc[:, 1:].values     # 데이터샘플
y = df.iloc[:, 0].values      # 타겟레이블

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1, stratify=y)

images = ['T-shirt', 'Trouser', 'Sweater', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Boot']


In [66]:
import cv2

hog = cv2.HOGDescriptor((28, 28), (14, 14), (7, 7), (7, 7), 9)

hog_x_train = np.zeros((5000,324),dtype="float64")
hog_x_test = np.zeros_like(hog_x_train)

for i in range(5000):
  hog_x_train[i]=hog.compute(x_train[i].reshape(28,28).astype(np.uint8)).squeeze()

for i in range(5000):
  hog_x_test[i]=hog.compute(x_test[i].reshape(28,28).astype(np.uint8)).squeeze()

x_train = hog_x_train
x_test = hog_x_test

In [67]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(x_train)
x_train = sc.transform(x_train)
x_test = sc.transform(x_test)


In [68]:
TOP=0
ELSE=1

corse_classify=np.ones_like(y_train)
test_corse=np.ones_like(y_test)

corse_classify[y_train==0]=TOP
corse_classify[y_train==2]=TOP
corse_classify[y_train==3]=TOP
corse_classify[y_train==4]=TOP
corse_classify[y_train==6]=TOP


test_corse[y_test==0]=TOP
test_corse[y_test==2]=TOP
test_corse[y_test==3]=TOP
test_corse[y_test==4]=TOP
test_corse[y_test==6]=TOP


In [69]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(weights='distance',algorithm='kd_tree',p=1,n_neighbors=3)#0.9854
knn.fit(x_train, corse_classify)
pred_y_knn = knn.predict(x_test)

print('knn 분류기 정확도', accuracy_score(test_corse, pred_y_knn))
'''params = {
    'n_neighbors': [3, 5,7,11,13,17],
    'weights': ['uniform', 'distance'],
    'algorithm':['auto','ball_tree','kd_tree','brute'],
    'p':[1,2]
}


grid_tree = GridSearchCV(knn, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, corse_classify)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
knn = grid_tree.best_estimator_
pred_y_knn = knn.predict(x_test)

#knn.fit(x_train, corse_classify)
#pred_y_knn = knn.predict(x_test)
print('knn 분류기 정확도', accuracy_score(test_corse, pred_y_knn))#Voting 분류기 정확도 0.99'''

'''best parameters :  {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
best score :  0.983399838639715
knn 분류기 정확도 0.986'''

knn 분류기 정확도 0.9854


"best parameters :  {'algorithm': 'auto', 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}\nbest score :  0.983399838639715\nknn 분류기 정확도 0.986"

In [70]:
from sklearn.neural_network import MLPClassifier

score = np.zeros((3,))
for i in range(3):
  mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', batch_size=500,
                      solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-2,random_state=i+1)
  mlp.fit(x_train, corse_classify)
  pred_y_mlp = mlp.predict(x_test)
  score[i]=accuracy_score(test_corse, pred_y_mlp)

print(f'mlp 분류기 정확도:{np.average(score):.3f}')
'''mlp = MLPClassifier()

params = {
    'hidden_layer_sizes': [(100,),(100,10),(512,128,32),(512,128),(20,),(20,2)],
    'activation': ['relu'],
    'batch_size':[500,100,50],
    'solver':['adam'],
    'learning_rate':['adaptive','invscaling'],
    'max_iter':[500,250],
    'alpha':[1e-2,1e-1,1e0,1e1,1e2]
}


grid_tree = GridSearchCV(mlp, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, corse_classify)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
mlp = grid_tree.best_estimator_

pred_y_mlp = mlp.predict(x_test)

#mlp.fit(x_train, corse_classify)
#pred_y_mlp = mlp.predict(x_test)
print('mlp 분류기 정확도', accuracy_score(test_corse, pred_y_mlp))#Voting 분류기 정확도 0.99'''
'''
best parameters :  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 500, 'hidden_layer_sizes': (100, 10), 'learning_rate': 'adaptive', 'max_iter': 500, 'solver': 'adam'}
best score :  0.987999758991779
mlp 분류기 정확도 0.9878
'''

mlp 분류기 정확도:0.989


"\nbest parameters :  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 500, 'hidden_layer_sizes': (100, 10), 'learning_rate': 'adaptive', 'max_iter': 500, 'solver': 'adam'}\nbest score :  0.987999758991779\nmlp 분류기 정확도 0.9878\n"

In [71]:
from sklearn import svm

score = np.zeros((3,))
print(x_train.shape)
for i in range(3):
  svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1,random_state=i+1)
  svm_clf.fit(x_train, corse_classify)
  pred_y_svm = svm_clf.predict(x_test)
  score[i]=accuracy_score(test_corse, pred_y_svm)
print(score)
print(f'mlp 분류기 정확도:{np.average(score):.3f}')#0.992


'''from sklearn import svm
svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)#0.9878
#svm_clf = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1)

svm_clf = svm.SVC()

params = {
    'kernel': ['poly','rbf'],
    'gamma': ['auto',1e-2,1e-1,1e0,1e1,1e2],
    'C':[1e-2,1e-1,1e0,1e1,1e2],
    'tol':[1e-2,1e-1,1e0,1e1,1e-3],
    'probability':[True],
    'max_iter':[100,500,-1]
}


grid_tree = GridSearchCV(svm_clf, param_grid=params, cv=3, refit=True)
grid_tree.fit(x_train, corse_classify)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)
svm_clf = grid_tree.best_estimator_
pred_y_svm = svm_clf.predict(x_test)

#svm_clf.fit(x_train, corse_classify)
#pred_y_svm = svm_clf.predict(x_test)
print('svm_clf 분류기 정확도', accuracy_score(test_corse, pred_y_svm))#Voting 분류기 정확도 0.99'''

'''
best parameters :  {'C': 1.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 100, 'probability': True, 'tol': 0.01}
best score :  0.9849995187037023
svm_clf 분류기 정확도 0.9538
'''

(5000, 324)


KeyboardInterrupt: ignored

In [ ]:
'''index=np.arange(test_corse.shape[0])

idx_knn =index[test_corse != pred_y_knn]
idx_mlp=index[test_corse != pred_y_mlp]
idx_svm=index[test_corse != pred_y_svm]

idx = idx_knn
idx=np.append(idx,idx_mlp)
idx=np.append(idx,idx_svm)

u,c=np.unique(idx,return_counts=True)
com_idx = u[c>1]

print(f"knn 불일치 개수 {idx_knn.shape[0]}")
print(f"mlp 불일치 개수 {idx_mlp.shape[0]}")
print(f"svm 불일치 개수 {idx_svm.shape[0]}")

print(f"중복 불일치 개수 {com_idx.shape[0]}")


row=int(np.sqrt(com_idx.shape[0]))+1

plt.rcParams['figure.figsize'] = [15, 15] # [width, height] (inches)
for i in range(com_idx.shape[0]):
  plt.subplot(row,row,i+1),plt.imshow(x_train[com_idx[i]].reshape(28,-1),cmap='gray'),plt.title(images[y_train[com_idx[i]]])
plt.show()
'''

In [1]:
from sklearn.ensemble import VotingClassifier

knn = KNeighborsClassifier(weights='distance',algorithm='kd_tree',p=1,n_neighbors=3)

mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', batch_size=500,
                      solver='adam', learning_rate="adaptive",  max_iter=500,alpha=1e-2)

svm_clf_1 = svm.SVC(kernel='poly',gamma='auto',probability=True,C=1e1,tol=1e-1,random_state=1)
svm_clf_2 = svm.SVC(kernel='rbf',gamma='auto',probability=True,C=3e0,tol=1e-1,random_state=1)
vo_clf = VotingClassifier(estimators=[('svm1',svm_clf_1),('svm2',svm_clf_2)],  voting='soft')

vo_clf.fit(x_train, corse_classify)
pred_y = vo_clf.predict(x_test)
print('corse 분류기 정확도', accuracy_score(test_corse, pred_y))#Voting 분류기 정확도 0.99

NameError: ignored